<img src="https://julialang.org/assets/infra/logo.svg" alt="Julia" width="200" style="max-width:100%;">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/darenasc/mlj-tutorials/master)

# Julia programming language

The sintax in Julia is very intuitive and similar to languages such as Matlab and Python.

In [ ]:
print("Hello world!")

In [ ]:
2 + 2

In [ ]:
typeof(42.0)

## Type inference 

Type annotation is not necessary in the code but it makes the code readable. *Type inference* is the process of identifying the types of the arguments to dispatch the right method. The *multiple dispatch* Julia feature make programs more efficient.

In [ ]:
function function_x(x::String)
    println("this is a string: $x")
end

function function_x(x::Int)
    println("$(x^2) is the square of $x")
end

In [ ]:
function_x("a string")
function_x(2)

<img src="https://github.com/alan-turing-institute/MLJ.jl/raw/master/material/MLJLogo2.svg?sanitize=true" alt="MLJ" width="200" style="max-width:100%;">

# MLJ

MLJ (Machine Learning in Julia) is a toolbox written in Julia providing a common interface and meta-algorithms for selecting, tuning, evaluating, composing and comparing machine learning models written in Julia and other languages. MLJ is released under the MIT licensed and sponsored by the [Alan Turing Institute](https://www.turing.ac.uk/).

### The MLJ Universe

The functionality of MLJ is distributed over a number of repositories
illustrated in the dependency chart below.

[MLJ](https://github.com/alan-turing-institute/MLJ) * [MLJBase](https://github.com/alan-turing-institute/MLJBase.jl) * [MLJModelInterface](https://github.com/alan-turing-institute/MLJModelInterface.jl) * [MLJModels](https://github.com/alan-turing-institute/MLJModels.jl) * [MLJTuning](https://github.com/alan-turing-institute/MLJTuning.jl) * [MLJLinearModels](https://github.com/alan-turing-institute/MLJLinearModels.jl) * [MLJFlux](https://github.com/alan-turing-institute/MLJFlux.jl) * [MLJTutorials](https://github.com/alan-turing-institute/MLJTutorials) * [MLJScientificTypes](https://github.com/alan-turing-institute/MLJScientificTypes.jl) * [ScientificTypes](https://github.com/alan-turing-institute/ScientificTypes.jl)


<div align="center">
    <img src="https://github.com/alan-turing-institute/MLJ.jl/raw/master/material/MLJ_stack.svg?sanitize=true" alt="Dependency Chart">
</div>

*Dependency chart for MLJ repositories. Repositories with dashed
connections do not currently exist but are planned/proposed.*

MLJ provides access to to a wide variety of machine learning models. For the most up-to-date list of available models `models()`.

In [ ]:
using MLJ
models()

## Fit, predict, transform

The following example is using the `fit()`, `predict()`, and `transform()` functions of MLJ.

In [ ]:
import Statistics
using PrettyPrinting
using StableRNGs

In [ ]:
X, y = @load_iris;

let's also load the DecisionTreeClassifier:

In [ ]:
@load DecisionTreeClassifier
tree_model = DecisionTreeClassifier()

## MLJ Machine

In MLJ, a *model* is an object that only serves as a container for the hyperparameters of the model. A *machine* is an object wrapping both a model and data and can contain information on the *trained* model; it does *not* fit the model by itself. However, it does check that the model is compatible with the scientific type of the data and will warn you otherwise.

In [ ]:
tree = machine(tree_model, X, y)

A machine is used both for supervised and unsupervised model. In this tutorial we give an example for the supervised model first and then go on with the unsupervised case.

## Training and testing a supervised model

Now that you've declared the model you'd like to consider and the data, we are left with the standard training and testing step for a supervised learning algorithm.

## Splitting the data

To split the data into a training and testing set, you can use the function `partition` to obtain indices for data points that should be considered either as training or testing data:

In [ ]:
rng = StableRNG(566)
train, test = partition(eachindex(y), 0.7, shuffle=true, rng=rng)
test[1:3]

## Fitting and testing the machine

To fit the machine, you can use the function `fit!` specifying the rows to be used for the training:

In [ ]:
fit!(tree, rows=train)

Note that this **modifies** the machine which now contains the trained parameters of the decision tree. You can inspect the result of the fitting with the `fitted_params` method:

In [ ]:
fitted_params(tree) |> pprint

This `fitresult` will vary from model to model though classifiers will usually give out a tuple with the first element corresponding to the fitting and the second one keeping track of how classes are named (so that predictions can be appropriately named).

You can now use the machine to make predictions with the `predict` function specifying rows to be used for the prediction:

In [ ]:
ŷ = predict(tree, rows=test)
@show ŷ[1]

Note that the output is probabilistic, effectively a vector with a score for each class. You could get the mode by using the `mode` function on `ŷ` or using `predict_mode`:

In [ ]:
ȳ = predict_mode(tree, rows=test)
@show ȳ[1]
@show mode(ŷ[1])

To measure the discrepancy between ŷ and y you could use the average cross entropy:

In [ ]:
mce = cross_entropy(ŷ, y[test]) |> mean
round(mce, digits=4)

# A more advanced example

In [ ]:
using MLJ
using StableRNGs
import DataFrames
@load RidgeRegressor pkg=MultivariateStats

In this tutorial we will show how to generate a model from a network; there are two approaches:

* using the `@from_network` macro
* writing the model in full

the first approach should usually be the one considered as it's simpler.

Generating a model from a network allows subsequent composition of that network with other tasks and tuning of that network.

### Using the @from_network macro

Let's define a simple network

*Input layer*

In [ ]:
rng = StableRNG(6616) # for reproducibility
x1 = rand(rng, 300)
x2 = rand(rng, 300)
x3 = rand(rng, 300)
y = exp.(x1 - x2 -2x3 + 0.1*rand(rng, 300))
X = DataFrames.DataFrame(x1=x1, x2=x2, x3=x3)
test, train = partition(eachindex(y), 0.8);

Xs = source(X)
ys = source(y, kind=:target)

*First layer*

In [ ]:
std_model = Standardizer()
stand = machine(std_model, Xs)
W = MLJ.transform(stand, Xs)

box_model = UnivariateBoxCoxTransformer()
box = machine(box_model, ys)
z = MLJ.transform(box, ys)

*Second layer*

In [ ]:
ridge_model = RidgeRegressor(lambda=0.1)
ridge = machine(ridge_model, W, z)
ẑ = predict(ridge, W)

*Output*

In [ ]:
ŷ = inverse_transform(box, ẑ)

No fitting has been done thus far, we have just defined a sequence of operations.

To form a model out of that network is easy using the `@from_network` macro:

In [ ]:
@from_network CompositeModel(std=std_model, box=box_model,
                             ridge=ridge_model) <= ŷ;

The macro defines a constructor CompositeModel and attributes a name to the different models; the ordering / connection between the nodes is inferred from `ŷ` via the `<= ŷ`.

**Note**: had the model been probabilistic (e.g. `RidgeClassifier`) you would have needed to add `is_probabilistic=true` at the end.

In [ ]:
cm = machine(CompositeModel(), X, y)
res = evaluate!(cm, resampling=Holdout(fraction_train=0.8, rng=51),
                measure=rms)
round(res.measurement[1], sigdigits=3)

## You can check more [Data Science tutorials in Julia](https://alan-turing-institute.github.io/DataScienceTutorials.jl/).